In [ ]:
# データの永続化
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

In [ ]:
# ソースからのHuggingface Transformersのインストール
!git clone https://github.com/huggingface/transformers -b v4.4.2
!pip install -e transformers
!pip install fugashi[unidic-lite]
!pip install ipadic

In [ ]:
# メニュー「ランタイム → ランタイムを再起動」で「Google Colab」を再起動

# 作業フォルダに戻る
%cd '/content/drive/My Drive/work/'

In [ ]:
# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

In [ ]:
# MeCabのインストール
!pip install mecab-python3

In [ ]:
import json
import MeCab
wakati = MeCab.Tagger("-Owakati")

# paragraphのわかち書き
def wakatiParagraph(dic):
    context = None
    for k, v in dic.items():
        if k == 'context':
            dic[k] = wakati.parse(v).strip()
            context = dic[k]

    for k, v in dic.items():
        if k == 'qas':
            for qa in v:
                if  not wakatiQa(qa, context):
                    print('Warning Remove QA:', context)
                    v.remove(qa)


# qaのわかち書き
def wakatiQa(dic, context):
    for k, v in dic.items():
        if k == 'question':
            dic[k] = wakati.parse(v).strip()
        elif k == 'answers':
            for answer in v:
                answer['text'] = wakati.parse(answer['text']).strip()
                answer['answer_start'] = context.find(answer['text'])
                if answer['answer_start'] < 0:
                    print('Warning Remove Text: ', answer['text'])
                    v.remove(answer)
                    if len(v) == 0:
                        return False
    return True


# ファイルのわかち書き
def wakatiFile(in_file, out_file):
    data = None
    with open(in_file, 'r') as f:
        data = json.load(f)
        paragraphs = data['data'][0]['paragraphs']
        for paragraph in paragraphs:
            wakatiParagraph(paragraph)

    with open(out_file, 'w') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

# ファイルのわかち書きの実行
wakatiFile('DDQA-1.0_RC-QA_train.json', 'wakati_DDQA-1.0_RC-QA_train.json')
wakatiFile('DDQA-1.0_RC-QA_dev.json', 'wakati_DDQA-1.0_RC-QA_dev.json')

In [ ]:
%%time

# ファインチューニングの実行
!python ./transformers/examples/legacy/question-answering/run_squad.py \
    --model_type=bert \
    --model_name_or_path=cl-tohoku/bert-base-japanese-whole-word-masking \
    --do_train \
    --do_eval \
    --max_seq_length=384 \
    --per_gpu_train_batch_size=12 \
    --learning_rate=3e-5 \
    --num_train_epochs=3 \
    --train_file=wakati_DDQA-1.0_RC-QA_train.json \
    --predict_file=wakati_DDQA-1.0_RC-QA_dev.json \
    --output_dir=output/ \
    --overwrite_output_dir

In [ ]:
# 学習状況の確認
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
import torch
from transformers import BertJapaneseTokenizer, AutoModelForQuestionAnswering
import MeCab
wakati = MeCab.Tagger("-Owakati")

# トークナイザーとモデルの準備
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking') 
model = AutoModelForQuestionAnswering.from_pretrained('output/')  

# コンテキストと質問
context = wakati.parse('日曜日に友達と秋葉原に遊びに行きました。').strip()
question = wakati.parse('どこに遊びに行った？').strip()
print(wakati.parse('日曜日に友達と秋葉原に遊びに行きました。'), wakati.parse('どこに遊びに行った？'))

# テキストをテンソルに変換
inputs = tokenizer.encode_plus(question, context, return_tensors='pt')

# 入力のトークンIDの配列の取得
input_ids = inputs['input_ids'].tolist()[0]

# 推論
model.eval()
with torch.no_grad():
    output = model(**inputs)
    answer_start = torch.argmax(output.start_logits)  
    answer_end = torch.argmax(output.end_logits) + 1 
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    print(answer)

In [ ]:
%%time

# SQUADによるファインチューニングの実行
!python ./transformers/examples/question-answering/run_qa.py \
    --model_name_or_path=bert-base-uncased \
    --dataset_name=squad \
    --do_train \
    --do_eval \
    --per_device_train_batch_size=12 \
    --learning_rate=3e-5 \
    --num_train_epochs=3 \
    --max_seq_length=384 \
    --output_dir=output/ \
    --overwrite_output_dir